In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [2]:
## Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS','AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])

In [3]:
CRIME_IDX = 0
ZX_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

# property_stats = np.ndarray(shape=(1,11))
# property_stats[0][CRIME_IDX] = features['CRIM'].mean()
# property_stats[0][ZX_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()

## avg value off every feature
property_stats = features.mean().values.reshape(1,11)
## features.mean() = Series
## features.mean().values = 1-D array

In [4]:
regr = LinearRegression().fit(features,target)
fitted_vals = regr.predict(features)

MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [5]:
def get_log_estimate(nr_rooms, students_per_classroom, next_to_river=False,
                    high_confidence=True):
    
    ## Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
        
    ## Make prediction
    log_estimates = regr.predict(property_stats)[0][0]
    
    ## Range
    if high_confidence:
        upper_bound = log_estimates + 2*RMSE
        lower_bound = log_estimates - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimates + RMSE
        lower_bound = log_estimates - RMSE
        interval = 68
    
    return log_estimates, upper_bound, lower_bound, interval

In [6]:
get_log_estimate(3,20,next_to_river=True, high_confidence=False)

(2.776758191480399, 2.9642703266775294, 2.589246056283269, 68)

In [7]:
get_log_estimate(3,20,next_to_river=True, high_confidence=True)

(2.776758191480399, 3.1517824618746597, 2.4017339210861386, 95)

In [8]:
np.median(boston_dataset.target)

21.2

In [12]:
## Converting log prices to todays price

ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9,15,next_to_river=False,
                                        high_confidence=False)

# Convert to todays $
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round $ to thousands
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f'At {conf}% confidence')
print(f'Estimated property value = {rounded_est}$')
print(f'{rounded_low}$ at lower end to {rounded_hi}$ at high end')




At 68% confidence
Estimated property value = 827000.0$
685000.0$ at lower end to 997000.0$ at high end


In [10]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    
    """Estimate the price of property in boston
    
    Keywords:
    rm -- no. of rooms
    ptratio -- no. of students per teacher in a class
    chas -- True for property next to river or False
    large_range -- True for 95% prediction, False for 65%
    
    """
    
    if rm < 1 or ptratio < 1:
        print('Unrealistic. Try again')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm,
                                        students_per_classroom=ptratio,
                                        next_to_river=chas,
                                        high_confidence=large_range)

    # Convert to todays $
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round $ to thousands
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'At {conf}% confidence')
    print(f'Estimated property value = {rounded_est}$')
    print(f'{rounded_low}$ at lower end to {rounded_hi}$ at high end')



In [11]:
get_dollar_estimate(rm=1, ptratio=0, chas=True)

Unrealistic. Try again
